作业第5周 循环神经网络练习
1.仿照课件关于IMDb数据集的分类训练，在课件示例基础上改用GRU、优化网络层数与其它参数，尽力提升分类准确率。
要求：使用callback方法保存最佳模型；训练完成后将保存在文件的模型读入，用于评估与预测测试集样本。
请每个人在"save"目录下建立以自己学号命名的子目录，然后在该子目录下保存文件

In [ ]:
from tensorflow import keras
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import numpy as np

max_features = 20000
max_len = 380

(x_train,y_train),(x_val,y_val) = imdb.load_data(num_words=max_features)

In [ ]:
print('training sequences:',len(x_train))
print('validation sequences:',len(x_val))

training sequences: 25000
validation sequences: 25000


In [4]:
word_index = imdb.get_word_index()
# word_index

In [5]:
index_to_word = {v:k for k,v in word_index.items()}
# index_to_word

In [6]:
''.join([index_to_word[x] for x in x_train[0]])

"theasyouwithoutthemselvespowerfulletslovestheirbecomesreachinghadjournalistoflotfromanyonetohaveafteroutatmospherenevermoreroomanditsoheartshowstoyearsofeverynevergoingandhelpmomentsorofeverychestvisualmovieexceptherwasseveralofenoughmorewithisnowcurrentfilmasyouofminepotentiallyunfortunatelyofyouthanhimthatwithoutthemselveshergetforwascampofyoumoviesometimesmoviethatwithscarybutpratfallstostorywonderfulthatinseeingincharactertoof70smusicianswithhearthadshadowstheyofherethatwithherserioustohavedoeswhenfromwhywhathavecriticstheyisyouthatisn'tonewillverytoasitselfwithothertrickyinofseenoverlandedforanyoneofandbrshow'stowhetherfromthanoutthemselveshistoryhenamehalfsomebrof'noddwastwomostofmeanfor1anyanboatsheheshouldisthoughtfrogbutofscriptyounotwhilehistoryhehearttorealatbarrelbutwhenfromonebitthenhavetwoofscripttheirwithhernobodymostthatwithwasn'ttowitharmedactingwatchanforwithheartfeltfilmwantan"

In [9]:
from keras_preprocessing import sequence
x_train_pad = sequence.pad_sequences(x_train,maxlen=max_len)
x_val_pad = sequence.pad_sequences(x_val,maxlen=max_len)

In [12]:
x_train_pad[0].shape,y_train[0]

((380,), 1)

In [10]:
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation
from tensorflow.keras.layers import Embedding,Bidirectional
from tensorflow.keras.layers import GRU
from tensorflow.keras.optimizers import SGD,Adam

In [ ]:
model = Sequential()

model.add(Embedding(output_dim=64,input_dim=max_features,
                    input_length=max_len))
forward_gru = GRU(units=128,dropout=0.2,recurrent_dropout=0.2,
                            return_sequences=True)
model.add(Bidirectional())
model.add(Dense(units=64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=1,activation='sigmoid'))

model.summary()

In [ ]:
opt = Adam(learning_rate=1e-4,decay=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=opt,metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint
file_path = '/save/3200102349/imdb_by_bigru.h5'
ModelCheckpoint(filepath=file_path,monitor='val_accuracy',
                verbose=0,save_best_only=True)

In [ ]:
train_history = model.fit(x_train_pad,y_train,
                          batch_size=64,epochs=20,verbose=1,
                          validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('train history')
    plt.ylabel(train)
    plt.xlabel('epoch')
    plt.legend(['train','validation'])
    plt.show()

show_train_history(train_history,'accuracy','val_accuracy')
show_train_history(train_history,'loss','val_loss')

In [ ]:
from keras.models import load_model
model = load_model(filepath=file_path)

scores = model.evaluate(x_val_pad,y_val,verbose=1)
scores[1]